# Projeto 2
O projeto consiste na detecção e equalização do ritmo alpha. Na base de dados, o ritmo alpha representa sinais com amplitude na faixa de 20 a 200 V e frequência entre 8 e 12 Hz, sendo melhor detectado sobre a região occipital, aparecendo com maior intensidade em indivíduos em repouso e com os olhos fechados.

In [4]:
from mne.time_frequency import psd_welch as pw
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import mne

### Carregamento do arquivo `OpenBCI-RAW-2019-06-07_01-09-36.txt`

In [5]:
def openData(name):
    df = pd.read_csv(name, skiprows=6, names=['index', 'PO3', 'PO4', 'P8', 'O1', 'O2', 'P7', '7', '8', 'x', 'y', 'z', 'tempo'])
    df = df.drop(['index','7', '8', 'x', 'y', 'z', 'tempo'], axis=1)
    dataLen = len(df)
    return dataLen, df.transpose()

### Verificando os tempos da evocação do ritmo alpha (em ~10 Hz) com a ferramenta `OpenBCI GUI 4.0.3`.

A seguir, encontram-se os intervalos dos tempos nos quais foram observados, via `OpenBCI`, que o ritmo alpha se encontra elevado em comparação com os demais.

098-101s&nbsp;&nbsp;&nbsp;&nbsp;128-130s&nbsp;&nbsp;&nbsp;&nbsp;135-137s&nbsp;&nbsp;&nbsp;&nbsp;142-148s&nbsp;&nbsp;&nbsp;&nbsp;152-158s  
`163-170s`172-176s&nbsp;&nbsp;&nbsp;&nbsp;182-192s&nbsp;&nbsp;&nbsp;&nbsp;194-198s&nbsp;&nbsp;&nbsp;&nbsp;252-254s  
263-265s&nbsp;&nbsp;&nbsp;&nbsp;271-278s&nbsp;&nbsp;&nbsp;&nbsp;286-291s&nbsp;&nbsp;&nbsp;&nbsp;292-297s&nbsp;&nbsp;&nbsp;&nbsp;299-301s  
302-307s&nbsp;&nbsp;&nbsp;&nbsp;332-334s&nbsp;&nbsp;&nbsp;&nbsp;335-339s&nbsp;&nbsp;&nbsp;&nbsp;411-413s&nbsp;&nbsp;&nbsp;&nbsp;418-421s  
428-432s&nbsp;&nbsp;&nbsp;&nbsp;442-445s&nbsp;&nbsp;&nbsp;&nbsp;446-450s&nbsp;&nbsp;&nbsp;&nbsp;460-462s&nbsp;&nbsp;&nbsp;&nbsp;465-467s  
479-482s&nbsp;&nbsp;&nbsp;&nbsp;484-486s&nbsp;&nbsp;&nbsp;&nbsp;490-494s&nbsp;&nbsp;&nbsp;&nbsp;500-503s&nbsp;&nbsp;&nbsp;&nbsp;510-512s  

### Após carregado o arquivo, este é transformado para o domínio do tempo.

In [16]:
def getRaw(df):
    ch_types = ['eeg'] * 6
    ch_names = ['PO3', 'PO4', 'P8', 'O1', 'O2', 'P7']
    info = mne.create_info(ch_names= ch_names, sfreq=256 , ch_types=ch_types)
    raw = mne.io.RawArray(df,info)
    montage = mne.channels.read_montage('standard_1020')
    raw.set_montage(montage)
    return raw

### Realizaando o conjunto de pré-processamento nos dados

O pré-processamento foi dividido em dois passos para remoção de ruidos dos dados e faixa de frequências não relevantes para o estudo.
- `Passo 1:` aplicar o filtro notch; e
- `Passo 2:` aplicar o filtro passa faixa no intervalo de 5 a 50 três vezes.

In [17]:
def preprocessing(raw):
    raw.notch_filter(np.arange(60, 121, 60), fir_design='firwin') #aplica filtro notch
    raw.filter(5,50) #aplica filtro passa-faixa
    raw.filter(5,50) #aplica filtro passa-faixa
    raw.filter(5,50) #aplica filtro passa-faixa
    return raw

### Visualização dos Dados

In [22]:
def plotGraphic(averageFrequency):
    ASD = averageFrequency
    y= ASD.keys()
    x= ASD.values()
    plt.bar(y, x)
    plt.pause(1.00)
    plt.clf()

### Foi realizado a plotagem (no domínio da frequência) para os trechos de tempos verificados dos sinais carregados através do `OpenBCI`.

A imagem abaixo é um exemplo da saída e se refere ao tempo `165-170s` que foi encontrado através do `OpenBCI` e gerado através do nosso algoritmo.

![Alfa](./diff_alfa.jpeg)

### Executando

Foi extraido os momentos corretos (frações de tempo) em que ocorre o ritmo alpha alevado frente aos demais sinais. Então, foi simulado a leitura do sinal na mesma frequência de amostragem que foi gravado (250 Hz), criando buffers de `n` segundos para a equalização do ritmo alpha em uma janela deslizante de tempo 1 segundo. Assim, foi gerado os valores do ritmo alpha em uma escala de 0 à 100, no qual o seu valor escalar é impresso na saída para cada deslizamento de tempo.

In [18]:

def work(raw, dataLen, bufferSize = 3):
    frequency = {
        'alpha'  : {},
        'beta'  : {},
        'gamma' : {},
        'theta' : {}
    }

    interval = {
        'alpha'  : {'init': 8, 'end': 12},
        'gamma' : {'init': 25, 'end': 100},
        'beta'  : {'init': 12, 'end': 30},
        'theta' : {'init': 5, 'end': 7}
    }

    for i in range(0, int(dataLen/256)):
        #realiza a janela de deslocamento (1 seg)
        psds, freqs = pw(raw,fmin=5,fmax=50,tmin=i,tmax=i+bufferSize)

        #recebe a maior média(dos 6 eletrodos) de cada frequencia em seu intervalo
        for canal in frequency.keys():
            frequency[canal] = max(np.mean(psds[:,interval[canal]['init']:interval[canal]['end']],axis=1))
            
        #verifica se a media do alpha é o maior
        if(frequency['alpha'] == max(frequency.values())):
            plotGraphic(frequency)        
            #hordena a medias
            values = sorted(list(frequency.values()) , reverse = True)
            #retorna a diferença do alpha sobre o segundo maior em escala de (0..100)
            poweralpha = 100*(values[0] - values[1])/frequency['alpha']
            print(poweralpha)

## Execução

In [23]:
dataLen, df = openData("entrada.csv")
raw = preprocessing(getRaw(df))
work(raw, dataLen, bufferSize = 5)

Setting up band-stop filter
Filter length of 1691 samples (6.605 sec) selected
Setting up band-pass filter from 5 - 50 Hz
l_trans_bandwidth chosen to be 2.0 Hz
h_trans_bandwidth chosen to be 12.5 Hz
Filter length of 423 samples (1.652 sec) selected
Setting up band-pass filter from 5 - 50 Hz
l_trans_bandwidth chosen to be 2.0 Hz
h_trans_bandwidth chosen to be 12.5 Hz
Filter length of 423 samples (1.652 sec) selected
Setting up band-pass filter from 5 - 50 Hz
l_trans_bandwidth chosen to be 2.0 Hz
h_trans_bandwidth chosen to be 12.5 Hz
Filter length of 423 samples (1.652 sec) selected
